# F1 Lap-by-Lap Race Prediction 3: pit stops
Data from http://ergast.com/mrd/db/  

This model uses an LSTM to predict the positions, laptimes and pitstops 20 drivers. 

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import os
import torch, torch.nn as nn, torch.optim as optim, torch.nn.functional as F
import math


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
db_dir = '/content/drive/My Drive/f1ml/db/'

In [4]:
# The time format in the Ergast database is MM:SS.ms
def time_to_int(t):
  if (t == float):
    return t
  t2 = str(t)
  ts = t2.rsplit(':')
  if ('\\N' in t2):
    return None
  if (not '.' in t2):
    return None
  if (len(ts) > 1):
    return int(ts[0]) * 60 + float(ts[1])
  else:
    return float(ts[0])

### Helper functions for getting information from ids

In [4]:
races = pd.read_csv(db_dir + 'races.csv')
circuits = pd.read_csv(db_dir + 'circuits.csv')
drivers = pd.read_csv(db_dir + 'drivers.csv')
constructor = pd.read_csv(db_dir + 'constructors.csv')
status = pd.read_csv(db_dir + 'status.csv')

def race_info(raceId):
  _races = races
  _r = _races.query(f'raceId  == {raceId}')
  if (_r.empty):
    return None, None, None
  _year = _r['year'].item()
  _round = _r['round'].item()
  _circuitId = _r['circuitId'].item()
  return _year, _round, _circuitId

def circuit_info(circuitId):
  _circuits = circuits
  _c = _circuits.query(f'circuitId  == {circuitId}')
  if (_c.empty):
    return None, None, None
  _name = _c['name'].item()
  _location = _c['location'].item()
  _country = _c['country'].item()
  return _name, _location, _country

def driver_info(id):
  _drivers = drivers
  _d = _drivers.query(f'driverId  == {id}')
  if (_d.empty):
    return None, None, None, None, None, None
  _number = _d['number'].item()
  _code = _d['code'].item()
  _forename = _d['forename'].item()
  _surname = _d['surname'].item()
  _dob = _d['dob'].item()
  _nationality = _d['nationality'].item()
  return _number, _code, _forename, _surname, _dob, _nationality

def constructor_info(id):
  _constructor = constructors
  _c = _constructor.query(f'constructorId  == {id}')
  if (_c.empty):
    return None, None
  _name = _d['name'].item()
  _nationality = _d['nationality'].item()
  return _name, _nationality

def status_info(id):
  _status = status
  _s = _status.query(f'statusId == {id}')
  if (_s.empty):
    return None
  _sstr = _s['status'].item()
  return _sstr

## Data preparation

Each csv file contains information of one race, and each row contains information of a lap. 

We are using the files prepared by the first notebook.  

In [6]:
years = [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
for year in years:
  if not os.path.exists(db_dir + f'races_npy/{year}'):
        os.makedirs(db_dir + f'races_npy/{year}')

### Status Embedding

In [5]:
stat_emb = [
  [4.0, 3.0, 130.0], # Accident/Collision
  [22.0, 5.0, 10.0, 23.0, 44.0, 47.0, 30.0, 32.0, 8.0, 38.0, 43.0, 85.0, 9.0, 86.0, 6.0, 2.0, 7.0, 87.0, 71.0, 41.0, 46.0, 37.0, 65.0, 78.0, 25.0, 74.0, 75.0, 26.0, 51.0, 40.0, 79.0, 36.0, 83.0, 80.0, 21.0, 69.0, 72.0, 70.0, 27.0, 60.0, 63.0, 29.0, 64.0, 66.0, 56.0, 59.0, 61.0, 42.0, 39.0, 48.0, 49.0, 34.0, 35.0, 28.0, 24.0, 33.0, 129.0, 76.0, 91.0, 131.0, 101.0, 132.0, 135.0,  84.0,  136.0,  105.0,  137.0,  138.0,  139.0], # Car issues
  [11.0,  13.0,  12.0,  14.0,  17.0,  15.0,  16.0, 18.0,  55.0,  58.0,  45.0, 88.0], # Lapped
  [0.0], # No problem
  [77.0, 73.0, 82.0, 81.0, 62.0, 54.0, 31.0, 96.0], # Other
  [20.0] #'Spun off'
] 

def stat_embed(id):
  _emb = np.zeros(6)
  for i in range(6):
    if id in stat_emb[i]:
      _emb[i] = 1
      return _emb
  _emb[4] = 1
  return _emb # Other

def stat_unbed(array, retired=False):
  _a = np.copy(array)
  if (retired):
    _a[3] = 0
  _i = np.argmax(_a)
  if (_i == 0):
    return 'Accident/Collision'
  elif (_i == 1):
    return 'Car Issues'
  elif (_i == 2):
    return 'Lapped'
  elif (_i == 3):
    return 'No Problem'
  elif (_i == 4):
    return 'Other'
  elif (_i == 5):
    return 'Spun off'
  else:
    return 'something is wrong'

### driverId embedding

In [6]:
drivers_short = pd.read_csv(db_dir + 'drivers_short.csv')
# from driverId to our id
def driver_embed_idx(driverId):
  row = drivers_short.query(f'driverId == {driverId}').index
  if (row.empty):
    return 0
  return row.item() + 1

# from our id to driverId
def driver_unbed_idx(idx):
  row = drivers_short.iloc[idx-1]
  return row['driverId']

# from our id to array
def driver_embed(idx):
  _e = np.zeros(130)
  _e[idx-1] = 1
  return _e

# from array to our id
def driver_unbed(d_array):
  return np.argmax(d_array) + 1

### New pit stop rep

In [ ]:
years = [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
for y in years:
  if not os.path.exists(db_dir + f'races/{y}'):
      os.makedirs(db_dir + f'races/{y}')

  cur_year = os.listdir(db_dir + f'races_v4/{y}/')
  for r in cur_year:
    if os.path.exists(db_dir + f'races/{y}/{r}'):
      continue
    
    cur_race = pd.read_csv(db_dir + f'races_v4/{y}/{r}')
    for j in range(20):
      for i in range(len(cur_race) - 1):
        if (cur_race[f'pitting{j+1}'][i] == 1):
          for k in range(i):
            if (cur_race[f'pitting{j+1}'][i-k-1] == 1):
              break
            cur_race[f'pitting{j+1}'][i-k-1] = k+2

    cur_race.drop('Unnamed: 0', axis=1)
    cur_race.to_csv(db_dir + f'races/{y}/{r}', index=False)
  print(y)
    


### embed to npy

In [28]:
years = [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
for year in years:
  if not os.path.exists(db_dir + f'races_npy/{year}'):
      os.makedirs(db_dir + f'races_npy/{year}')

  cur_year = os.listdir(db_dir + f'races/{year}/')
  counter = 0
  for round in cur_year:
    
    cur_race = pd.read_csv(db_dir + f'races/{year}/{round}')

    #if not os.path.exists(db_dir + f'races_npy/{year}/{round}_in.npy'):
    if (True):
      for i in range(len(cur_race)-1):
        # Size([141])
        cur = cur_race.iloc[i].values[1:142] # removes index and trim size
        # replace NaN with 0
        cur[cur != cur] = 0
        cur_ret = np.zeros(130)
        cur_ret[int(cur[0])] = 1 # circuitId
        lap_percentage = float(i) / (len(cur_race)-1)
        _lp = np.zeros(1)
        _lp[0] = lap_percentage
        cur_ret = np.append(cur_ret, _lp)
        for k in range(1, 141, 7):
    
          d_id = driver_embed(driver_embed_idx(int(cur[k])))
          cur_ret = np.append(cur_ret, d_id)
    
          # 2 bits: top or top3 or top10
          d_s = np.zeros(3)
          _s = int(cur[k+1])-1
          if (_s <= 1):
            d_s[0] = 1
          if (_s <=3):
            d_s[1] = 1
          if (_s <=10):
            d_s[2] = 1
          cur_ret = np.append(cur_ret, d_s)
    
          c_s = np.zeros(2)
          _c = int(cur[k+2])-1
          if (_c <= 1):
            c_s[0] = 1
          if (_c <= 3):
            c_s[1] = 1
          cur_ret = np.append(cur_ret, c_s)
    
          pos = np.zeros(21)
          _p = int(cur[k+3])-1
          if (_p >= 21):
            _p = 20
          pos[_p] = 1
          cur_ret = np.append(cur_ret, pos)
    
          pit = np.zeros(1)
          if (cur[k+4] == 1):
            pit[0] = 1
          else :
            pit[0] = 0
          cur_ret = np.append(cur_ret, pit)
    
          stat = stat_embed(int(cur[k+5]))
          cur_ret = np.append(cur_ret, stat)
    
          laptime = np.zeros(1)
          laptime[0] = float(cur[k+6]) / 100 # scaling
          cur_ret = np.append(cur_ret, laptime)
    
          rando = np.zeros(1)
          cur_ret = np.append(cur_ret, rando)
  
        if (i == 0):
          cur_race_in = cur_ret
        else:
          cur_race_in = np.vstack([cur_race_in, cur_ret])
      np.save(db_dir + f'races_npy/{year}/{counter}_in.npy', cur_race_in)
  
    #if not os.path.exists(db_dir + f'races_npy/{year}/{round}_exp.npy'):
    if (True):
      for i in range(len(cur_race)-1):
        next = cur_race.iloc[i+1].values[1:142] # removes index and trim size
        # replace NaN with 0
        next[next != next] = 0
        for k in range(1, 141, 7):
          pos = np.zeros(21)
          _p = int(next[k+3])-1
          if (_p >= 21):
            _p = 20
          pos[_p] = 1
          if (k == 1):
            next_ret = pos
          else:
            next_ret = np.append(next_ret, pos)
    
          pit = np.zeros(1)
          pit[0] = float(next[k+4]) / (len(cur_race)-1)
          next_ret = np.append(next_ret, pit)
    
          stat = stat_embed(int(next[k+5]))
          next_ret = np.append(next_ret, stat)
    
          laptime = np.zeros(1)
          laptime[0] = float(next[k+6]) / 100 # scaling
          next_ret = np.append(next_ret, laptime)

        if (i == 0):
          exp = next_ret
        else:
          exp = np.vstack([exp, next_ret])
      np.save(db_dir + f'races_npy/{year}/{counter}_exp.npy', exp)

    counter += 1
  print(year)
  

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [30]:
t = np.load(db_dir + 'races_npy/2001/0_exp.npy')
tt = t.transpose()

In [31]:
len(tt[0])

580

In [95]:
t = np.load(db_dir + 'races_npy/2001/0_in.npy')
t

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [96]:
len(t[0])

3431

## Dataset



Uses 'db/races_npy_v2/'  
v1:
Each data sample is a very long vector (3431) with the following features (and sizes):
- circuit id (130)
- lap number / total number of laps
- for each driver:
- - driver id (130)
- - driver standing (3)
- - constructor standing (2)
- - position (21)
- - pitting this lap OR laps/total number of laps till pitting (1) (input /output) (v1.1)
- - status (6)
- - laptime (1)
- - random (1)



In [15]:
# to make feature size consistent, we use laptimes of top 20 drivers
class RaceDataset(torch.utils.data.Dataset):
  def __init__(self, dir):

    self.dir = dir
    self.year = 2001
    self.round = 1
    self.cache = {}

    # number of races this year
    self.cur_year = int(len(os.listdir(self.dir + f'{self.year}/'))/2)


    if (self.round < self.cur_year):
      self.cur_race = np.load(self.dir + f'{self.year}/{self.round-1}_in.npy'), np.load(self.dir + f'{self.year}/{self.round-1}_exp.npy')
    else:
      self.cur_race = np.load(self.dir + f'{self.year}/{self.cur_year-1}_in.npy'), np.load(self.dir + f'{self.year}/{self.cur_year - 1}_exp.npy')
    self.cache[f'{self.year}_{self.round}'] = self.cur_race


  def set_year(self, year):
    if (year == 2021):
      return
    self.year = year
    self.cur_year = int(len(os.listdir(self.dir + f'{self.year}/'))/2)

  def set_round(self, round):
    self.round = round
    if (self.year == 2021):
      return # no data yet

    if (f'{self.year}_{self.round}' in self.cache):
      self.cur_race = self.cache[f'{self.year}_{self.round}']
      return
    # set round, if round > number of rounds in that season, set to the last round
    if (self.round < self.cur_year):
      self.cur_race = np.load(self.dir + f'{self.year}/{self.round-1}_in.npy'), np.load(self.dir + f'{self.year}/{self.round-1}_exp.npy')
    else:
      self.cur_race = np.load(self.dir + f'{self.year}/{self.cur_year-1}_in.npy'), np.load(self.dir + f'{self.year}/{self.cur_year - 1}_exp.npy')
    self.cache[f'{self.year}_{self.round}'] = self.cur_race

  def next_round(self):
    self.round += 1
    if (self.round < self.cur_year):
      self.set_round(self.round)
    else:
      self.round = 1
      self.set_year(self.year+1)
      self.set_round(self.round)


  def __len__(self):
    return len(self.cur_race[0])

  def __getitem__(self, i):
    cur_ret, next_ret = self.cur_race[0][i], self.cur_race[1][i]

    return (torch.from_numpy(cur_ret), torch.from_numpy(next_ret))

In [98]:
ds = RaceDataset(db_dir + 'races_npy/')


In [ ]:
ds[0]

In [100]:
len(ds[0][0])

3431

In [101]:
len(ds[0][1])

580

In [102]:
ds[0][1][29*11+28]

tensor(1.1545, dtype=torch.float64)

### Helper function for displaying dataset input/output

In [30]:
def pos_df(lap_in, out, num_of_laps=1):
  df = pd.DataFrame(columns=['code', 'driver', 'position', 'laps till pitting', 'status', 'laptime'])
  _lap = lap_in.detach().clone().numpy()
  _o = out.detach().clone().numpy()
  _name, _loc, _country = circuit_info(np.argmax(_lap[:130]))
  for i in range(20):
    _d_idx = driver_unbed_idx(driver_unbed(_lap[131 + i * 165 : 131 + i * 165 + 130]))
    _num, _code, _fn, _ln, _, _ = driver_info(_d_idx)
    _pos = np.argmax(_o[i*29 : i*29 + 21]) + 1
    _pitting = _o[i*29+21] * num_of_laps
    if (_pitting == 0):
      _pitting = 'NA'
    _retired = False
    if (_pos == 21):
      _retired = True
    _status = stat_unbed(_o[i*29 + 22: i*29 + 28], _retired)
    _time = _o[i*29 + 28] * 100
    df = df.append({
        'code': f'{_code}',
        'driver': f'{_fn} {_ln}',
        'position': _pos,
        'laps till pitting': _pitting,
        'status': _status,
        'laptime': _time
    }, ignore_index=True)

  df = df.sort_values(by=['position', 'laptime'], ascending=[True, False])
  return _name, _loc, _country, df


In [107]:
ds.set_year(2001)
ds.set_round(1)

In [108]:
_, _, _, df = pos_df(ds[1][0], ds[1][1])

In [109]:
df

,code,driver,position,laps till pitting,status,laptime
0,MSC,Michael Schumacher,1,NA,No Problem,93.098
2,\N,Mika Häkkinen,2,NA,No Problem,93.047
3,\N,Heinz-Harald Frentzen,3,NA,No Problem,94.128
1,BAR,Rubens Barrichello,4,NA,No Problem,92.853
5,COU,David Coulthard,5,NA,No Problem,93.344
6,TRU,Jarno Trulli,6,NA,No Problem,95.085
4,SCH,Ralf Schumacher,7,NA,No Problem,98.018
7,VIL,Jacques Villeneuve,8,NA,No Problem,96.364
9,HEI,Nick Heidfeld,9,NA,No Problem,96.104
8,\N,Olivier Panis,10,NA,No Problem,95.906


In [106]:
ds[0][0].shape

torch.Size([3431])

In [34]:
# Returns a tensor with the size of in but content of out
def out_to_in(in_, out_, random=False, num_of_laps=50):
  _ret = in_.detach().clone().numpy()
  _o = out_.detach().clone().numpy()
  _o = _o.reshape([580])
  _ret = _ret.reshape([3431])
  j = 0
  for i in range(0, 20):
    _ret[131 + i*165 + 135: i*165 + 131 + 164] = _o[j*29: (j+1) * 29]
    if (round(_o[j*29 + 21] * num_of_laps) <= 1):
      _ret[131 + i*165 + 130 + 26] = 1
    else:
      _ret[131 + i*165 + 130 + 26] = 0
    if (random):
      _ret[i * 165 + 131 + 164: (i+1) * 165 + 131] = np.random.uniform(0, 10, [1])
    j += 1

  return torch.from_numpy(_ret).float()


In [111]:
outfromin = out_to_in(ds[0][0], ds[0][1])

In [112]:
print(outfromin)

tensor([0., 1., 0.,  ..., 0., 0., 0.])


In [113]:
_, _, _, df2 = pos_df(outfromin, ds[0][1])


In [114]:
df2

,code,driver,position,laps till pitting,status,laptime
0,MSC,Michael Schumacher,1,NA,No Problem,97.197
2,\N,Mika Häkkinen,2,NA,No Problem,98.408
3,\N,Heinz-Harald Frentzen,3,NA,No Problem,99.631
4,SCH,Ralf Schumacher,4,NA,No Problem,101.300
1,BAR,Rubens Barrichello,5,NA,No Problem,101.450
6,TRU,Jarno Trulli,6,NA,No Problem,102.350
5,COU,David Coulthard,7,NA,No Problem,102.832
7,VIL,Jacques Villeneuve,8,NA,No Problem,103.367
9,HEI,Nick Heidfeld,9,NA,No Problem,104.263
8,\N,Olivier Panis,10,NA,No Problem,104.690


## Model

In [10]:
class RacePredictionModel(nn.Module):
    def __init__(self, input_size, output_size, lstm_hids, lstm_layers, dropout):
        super(RacePredictionModel, self).__init__()

        self.input_size = input_size
        self.lstm_layers = lstm_layers
        self.lstm_hids = lstm_hids

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=lstm_hids, num_layers=lstm_layers, dropout=dropout, batch_first=True)

        self.fc = nn.Linear(lstm_hids, output_size)

        nn.init.xavier_uniform_(self.fc.weight.data)
        for name, params in self.lstm.named_parameters():
            if name[:6] == 'weight':
                nn.init.xavier_uniform_(params)
            elif name[:4] == 'bias':
                nn.init.constant_(params, 0.0)

    def zero_states(self, batchsize=1):
        hidden_state = torch.zeros(self.lstm_layers, batchsize, self.lstm_hids)
        cell_state = torch.zeros(self.lstm_layers, batchsize, self.lstm_hids)
        return (hidden_state, cell_state)

    def forward(self, ins, prev_states=None):
        lstm_outs, next_states = self.lstm(ins, prev_states)
        outs = self.fc(lstm_outs)
        return outs, next_states

### Helper functions for training #1

In [21]:

# Trains model on data from 2001 to 2019, 2020 is reserved for testing
# Method 1
# Training procedure:
#   for each race:
#     set zero states,
#     for each lap:
#       feed input from dataset,
#       calculate loss from output,
#       back propagate,
def run_train(model, ds, crit, opt, sched):
    model.train()
    total_loss, total_count = 0, 0

    ds.set_year(2012)
    ds.set_round(1)

    while (ds.year != 2020):
      hid_state, cell_state = model.zero_states()
      states = hid_state.to(device), cell_state.to(device)
      for i in range(len(ds)):
          opt.zero_grad()
          lap_in, lap_exp = ds[i]
          #if (i == 0):
          #  lap_in, lap_exp = ds[i]
          #else:
          #  _, lap_exp = ds[i]
          #  lap_in = out_to_in(lap_in.cpu(), lap_out.cpu()).to(device)

          # fix shape and type
          # input size should be (batch, seq_len, input_size)
          # we are not using batches
          #if (i == 0):
          if (True):
            lap_in = lap_in.unsqueeze(0)
            lap_exp = lap_exp.unsqueeze(0)
          lap_in = lap_in.unsqueeze(0).float()
          lap_exp = lap_exp.unsqueeze(0).float()

          lap_in, lap_exp = lap_in.to(device), lap_exp.to(device)
          lap_out, states = model(lap_in, states)
          loss = crit(lap_out, lap_exp)
          for s in states:
              s.detach_()
          loss.backward()
          if (math.isnan(loss)):
            print('Loss is nan')
            print(f'Year: {ds.year}')
            print(f'round: {ds.round}')
            print(f'lap: {i}')
            print(lap_in)
            input()
          opt.step()
          total_loss += loss.item() * 1 #lap_in.size(0) # since we are not doing batches
          total_count += 1 #lap_in.size(0)
      ds.next_round()

    sched.step()
    return total_loss/total_count

def run_test(model, ds, crit):
    model.eval()
    total_loss, total_count = 0, 0

    ds.set_year(2020)
    ds.set_round(1)

    hid_state, cell_state = model.zero_states()
    states = hid_state.to(device), cell_state.to(device)

    for i in range(len(ds)):
      #lap_in, lap_exp = ds[i]
      if (i == 0):
        lap_in, lap_exp = ds[i]
      else:
        _, lap_exp = ds[i]
        lap_in = out_to_in(lap_in.cpu(), lap_out.cpu()).to(device)
      #if (i == 0):
      if (True):
        lap_in = lap_in.unsqueeze(0)
        lap_exp = lap_exp.unsqueeze(0)
      lap_in = lap_in.unsqueeze(0).float()
      lap_exp = lap_exp.unsqueeze(0).float()
      lap_in, lap_exp = lap_in.to(device), lap_exp.to(device)
      lap_out, states = model(lap_in, states)
      loss = crit(lap_out, lap_exp)
      total_loss += loss.item() * 1 #lap_in.size(0) # since we are not doing batches
      total_count += 1 #lap_in.size(0)

    return total_loss/total_count

def run_all(model, ds, crit, opt, sched, versionId, n_epochs=10):
    for epoch in tqdm(range(n_epochs), desc='epochs', unit='ep'):
      train_loss = run_train(model, ds, crit, opt, sched)
      test_loss = run_test(model, ds, crit)
      tqdm.write(f'epoch {epoch}   train loss {train_loss:.6f}  test loss {test_loss:.6f}')
      if epoch % 5 == 0:
        torch.save(model.state_dict(), f'/content/drive/My Drive/f1ml/sd/{versionId}-{epoch}-loss-{test_loss:.2f}.pth')
    torch.save(model.state_dict(), f'/content/drive/My Drive/f1ml/sd/{versionId}-{n_epochs}-loss-{test_loss:.2f}.pth')


### Helper functions for training #2

In [ ]:
# Trains model on data from 2001 to 2019, 2020 is reserved for testing
# Method 2
# Training procedure:
#   for each group of batches:
#     set zero states,
#     for each batch:
#       feed batch from loader,
#       calculate loss from output,
#       back propagate,
def run_train_2(model, dl, batchsize, crit, opt, sched):
    model.train()
    total_loss, total_count = 0, 0

    for batchnum in range(dl.num_batches):
      hid_state, cell_state = model.zero_states(batchsize)
      states = hid_state.to(device), cell_state.to(device)
      dl.reset()
      for i in range(dl.cur_batch_len):
        opt.zero_grad()
        lap_in, lap_exp = dl.get_batch()
  
        lap_in, lap_exp = lap_in.to(device), lap_exp.to(device)
        lap_out, states = model(lap_in, states)
        loss = crit(lap_out, lap_exp)
        for s in states:
            s.detach_()
        loss.backward()
        if (math.isnan(loss)):
          print('Loss is nan')
          print(f'Year: {ds.year}')
          print(f'round: {ds.round}')
          print(f'lap: {i}')
          print(lap_in)
          input()
        opt.step()
        total_loss += loss.item() * 1 #lap_in.size(0) # since we are not doing batches
        total_count += 1 #lap_in.size(0)

    sched.step()
    return total_loss/total_count

# same test
def run_test(model, ds, crit):
    model.eval()
    total_loss, total_count = 0, 0

    ds.set_year(2020)
    ds.set_round(1)

    hid_state, cell_state = model.zero_states()
    states = hid_state.to(device), cell_state.to(device)

    for i in range(len(ds)):
      lap_in, lap_exp = ds[i]
      #if (i == 0):
      if (True):
        lap_in = lap_in.unsqueeze(0)
        lap_exp = lap_exp.unsqueeze(0)
      lap_in = lap_in.unsqueeze(0).float()
      lap_exp = lap_exp.unsqueeze(0).float()
      lap_in, lap_exp = lap_in.to(device), lap_exp.to(device)
      lap_out, states = model(lap_in, states)
      loss = crit(lap_out, lap_exp)
      total_loss += loss.item() * 1 #lap_in.size(0) # since we are not doing batches
      total_count += 1 #lap_in.size(0)

    return total_loss/total_count

def run_all_2(model, ds, dl, batchsize, crit, opt, sched, versionId, n_epochs=10):
    for epoch in tqdm(range(n_epochs), desc='epochs', unit='ep'):
      train_loss = run_train_2(model, dl, batchsize, crit, opt, sched)
      test_loss = run_test(model, ds, crit)
      tqdm.write(f'epoch {epoch}   train loss {train_loss:.6f}  test loss {test_loss:.6f}')
      if epoch % 5 == 0:
        torch.save(model.state_dict(), f'/content/drive/My Drive/f1ml/sd/{versionId}-{epoch}-loss-{test_loss:.2f}.pth')
    torch.save(model.state_dict(), f'/content/drive/My Drive/f1ml/sd/{versionId}-{n_epochs}-loss-{test_loss:.2f}.pth')


### Training

In [36]:
# only init once, cause "caching"
ds = RaceDataset(db_dir + 'races_npy/')


In [49]:
device = torch.device('cuda:0')
#device = torch.device('cpu')
model = RacePredictionModel(3431, 580, 580, 2, 0.2)
model.to(device)
crit = nn.MSELoss().to(device)
opt = optim.Adam(model.parameters(), lr=0.001)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=0.2)
#dl = CustomeLoader(ds, 32)

In [45]:
run_all(model, ds, crit, opt, sched, 42, 11)
#run_all_2(model, ds, dl, 32, crit, opt, sched, 32, 10)

epoch 0   train loss 0.044126  test loss 0.072538
epoch 1   train loss 0.048533  test loss 0.054371
epoch 2   train loss 0.047186  test loss 0.055065
epoch 3   train loss 0.044693  test loss 0.052429
epoch 4   train loss 0.043669  test loss 0.053590
epoch 5   train loss 0.043407  test loss 0.054545
epoch 6   train loss 0.043323  test loss 0.054752
epoch 7   train loss 0.043320  test loss 0.054778
epoch 8   train loss 0.043312  test loss 0.054784
epoch 9   train loss 0.043316  test loss 0.054784
epoch 10   train loss 0.043298  test loss 0.054784



vId | training method | ds | ds cls| input size | hid size | layers | dropout | crit | opt | sched | epochs | train loss | test loss
--- | --- | --- | --- | --- | --- | --- |--- | --- | --- | --- | ---| --- | ---
39 | 1 | v1 | v1| 3431 | 580 | 2 | 0.2| MSE | Adam, lr=0.001 | StepLR, ss=1, g=0.2| 10|0.036|0.057  
40 | 1 | v1, from 2012 | v1| 3431 | 580 | 2 | 0.2| MSE | Adam, lr=0.001 | StepLR, ss=1, g=0.2| 10|0.0418|0.0567  
41 | 1 | v1.1, from 2012 | v1| 3431 | 580 | 2 | 0.2| MSE | Adam, lr=0.001 | StepLR, ss=1, g=0.2| 11|0.0397|0.0557  
42 | 1 | v1.1, from 2012 | v1| 3431 | 1160 | 2 | 0.2| MSE | Adam, lr=0.001 | StepLR, ss=1, g=0.2| 11|0.00433|0.0547  

In [ ]:
run_test(model, ds, crit)

0.06231908148653071

In [ ]:
model.eval()
ds.set_year(2020)
ds.set_round(20)
p, n = ds[10]
p = p.to(device)
hid_state, cell_state = model.zero_states()
states = (hid_state.to(device), cell_state.to(device))
out, s = model(p.unsqueeze(0).unsqueeze(0).float(), states)
out = out.squeeze().squeeze()
print(out)
pos_df(p.to(cpu), out.to(cpu))


In [50]:
cpu = torch.device('cpu')
#model.load_state_dict(torch.load('/content/drive/My Drive/f1ml/sd/34-5-loss-0.06.pth'))
model.load_state_dict(torch.load('/content/drive/My Drive/f1ml/sd/41-10-loss-0.06.pth'))
#model.load_state_dict(torch.load('/content/drive/My Drive/f1ml/sd/11-5-loss-37.31.pth'))
model.eval()
ds.set_year(2020)
ds.set_round(30)
p, n = ds[0]
p = p.to(device)
hid_state, cell_state = model.zero_states()
states = (hid_state.to(device), cell_state.to(device))
out, s = model(p.unsqueeze(0).unsqueeze(0).float(), states)
out = out.squeeze().squeeze()
for i in range(50):
  out, s = model(out_to_in(p.to(cpu), out.to(cpu), False).to(device).unsqueeze(0).unsqueeze(0).float(), s)
  out = out.squeeze().squeeze()
  a, b, c, d = pos_df(p.to(cpu), out.to(cpu), len(ds))
  print(a,b,c)
  display(d)
out = out.detach().to(cpu)
d

Yas Marina Circuit Abu Dhabi UAE


,code,driver,position,laps till pitting,status,laptime
0,VER,Max Verstappen,1,13.051569,No Problem,112.862444
1,BOT,Valtteri Bottas,2,12.789122,No Problem,110.156488
2,HAM,Lewis Hamilton,3,12.263862,No Problem,104.962647
3,NOR,Lando Norris,4,12.418055,No Problem,111.313295
4,ALB,Alexander Albon,5,12.178647,No Problem,109.002304
5,SAI,Carlos Sainz,6,10.489433,No Problem,109.551394
6,KVY,Daniil Kvyat,7,12.817556,No Problem,103.639412
7,STR,Lance Stroll,8,12.149128,No Problem,110.577428
9,OCO,Esteban Ocon,9,13.198343,No Problem,112.485206
8,GAS,Pierre Gasly,9,10.680150,No Problem,104.081535


Yas Marina Circuit Abu Dhabi UAE


,code,driver,position,laps till pitting,status,laptime
0,VER,Max Verstappen,1,11.159723,No Problem,113.304603
1,BOT,Valtteri Bottas,2,10.808966,No Problem,110.482931
2,HAM,Lewis Hamilton,3,10.951227,No Problem,106.217635
3,NOR,Lando Norris,4,11.041854,No Problem,111.284912
4,ALB,Alexander Albon,5,10.662400,No Problem,107.104957
6,KVY,Daniil Kvyat,5,11.330859,No Problem,101.770318
5,SAI,Carlos Sainz,6,9.114023,No Problem,110.713398
7,STR,Lance Stroll,7,10.383343,No Problem,113.540721
9,OCO,Esteban Ocon,8,12.064934,No Problem,113.962257
12,VET,Sebastian Vettel,9,12.345622,No Problem,112.084198


Yas Marina Circuit Abu Dhabi UAE


...

Yas Marina Circuit Abu Dhabi UAE


,code,driver,position,laps till pitting,status,laptime
0,VER,Max Verstappen,1,6.951587,No Problem,115.362728
1,BOT,Valtteri Bottas,2,6.701008,No Problem,115.288627
2,HAM,Lewis Hamilton,3,7.116453,No Problem,107.436621
5,SAI,Carlos Sainz,4,4.680083,No Problem,113.303280
3,NOR,Lando Norris,4,7.620010,No Problem,109.629917
7,STR,Lance Stroll,7,5.457018,No Problem,116.573620
9,OCO,Esteban Ocon,8,7.542816,No Problem,121.028280
12,VET,Sebastian Vettel,8,5.796774,No Problem,105.348754
11,LEC,Charles Leclerc,10,6.799624,No Problem,115.691590
13,GIO,Antonio Giovinazzi,12,5.445534,No Problem,109.140861


Yas Marina Circuit Abu Dhabi UAE


,code,driver,position,laps till pitting,status,laptime
0,VER,Max Verstappen,1,6.897869,No Problem,115.401459
1,BOT,Valtteri Bottas,2,6.648511,No Problem,115.335500
2,HAM,Lewis Hamilton,3,7.059718,No Problem,107.452500
5,SAI,Carlos Sainz,4,4.625269,No Problem,113.323355
3,NOR,Lando Norris,4,7.568447,No Problem,109.604931
7,STR,Lance Stroll,7,5.389007,No Problem,116.613770
9,OCO,Esteban Ocon,8,7.485137,No Problem,121.117067
12,VET,Sebastian Vettel,8,5.724645,No Problem,105.349171
11,LEC,Charles Leclerc,10,6.731765,No Problem,115.732861
13,GIO,Antonio Giovinazzi,12,5.373957,No Problem,109.114063


,code,driver,position,laps till pitting,status,laptime
0,VER,Max Verstappen,1,6.897869,No Problem,115.401459
1,BOT,Valtteri Bottas,2,6.648511,No Problem,115.335500
2,HAM,Lewis Hamilton,3,7.059718,No Problem,107.452500
5,SAI,Carlos Sainz,4,4.625269,No Problem,113.323355
3,NOR,Lando Norris,4,7.568447,No Problem,109.604931
7,STR,Lance Stroll,7,5.389007,No Problem,116.613770
9,OCO,Esteban Ocon,8,7.485137,No Problem,121.117067
12,VET,Sebastian Vettel,8,5.724645,No Problem,105.349171
11,LEC,Charles Leclerc,10,6.731765,No Problem,115.732861
13,GIO,Antonio Giovinazzi,12,5.373957,No Problem,109.114063


In [ ]:
pos_df(p.to(cpu), n.to(cpu))

('Yas Marina Circuit',
 'Abu Dhabi',
 'UAE',
    code              driver position  pitting      status  laptime
 0   VER      Max Verstappen        1      0.0  No Problem    103.0
 1   BOT     Valtteri Bottas        2      0.0  No Problem    105.0
 2   HAM      Lewis Hamilton        3      0.0  No Problem    106.0
 3   NOR        Lando Norris        4      0.0  No Problem    107.0
 4   ALB     Alexander Albon        5      0.0  No Problem    108.0
 5   SAI        Carlos Sainz        6      0.0  No Problem    109.0
 6   KVY        Daniil Kvyat        7      0.0  No Problem    110.0
 7   STR        Lance Stroll        8      0.0  No Problem    111.0
 9   OCO        Esteban Ocon        9      0.0  No Problem    112.0
 8   GAS        Pierre Gasly       10      0.0  No Problem    112.0
 10  RIC    Daniel Ricciardo       11      0.0  No Problem    113.0
 12  VET    Sebastian Vettel       12      0.0  No Problem    113.0
 11  LEC     Charles Leclerc       13      0.0  No Problem    114.0
 14

In [ ]:
c = torch.cat((ds[0][0].unsqueeze(0).unsqueeze(0), ds[1][0].unsqueeze(0).unsqueeze(0)), 0)

In [ ]:
c.shape

torch.Size([2, 1, 5330])

In [ ]:
o, s = model(c.to(device).float(), states)

RuntimeError: ignored

In [ ]:
o.shape

torch.Size([1, 2, 1600])

In [ ]:
s[0].shape

torch.Size([10, 1, 128])

In [ ]:
s[1].shape

torch.Size([10, 1, 128])

In [ ]:
oo, ss = model(ds[0][0].unsqueeze(0).unsqueeze(0).float().to(device), states)

In [ ]:
oo.shape

torch.Size([1, 1, 1600])

In [ ]:
ss[0].shape

torch.Size([10, 1, 128])

In [ ]:
len(ss)

2